# Running the Micom Workflow for the Binary Azotobacter - Rhodosporidium Model

In this notebook we utilize the package `micom` to generate a binary community model for 2 organisms of interest:
- `Azotobacter vinelandii`
- `Rhodosporidium toruloides`

This binary consortium allows us to gain insights into the exchanges between the 2 organisms and run FBA experiments.

First off we can import all necessary packages for this notebook.

In [1]:
import pandas as pd
import cobra

from micom import Community
from micom.workflows import build, grow, tradeoff, fix_medium, build_database
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

import os
os.environ["GRB_LICENSE_FILE"]

'C:\\\\Users\\\\pino216\\\\gurobi-3.lic'

This taxonomy file lists important information for `micom` down the road, such as the number of `reactions` and `metabolites` in the provided models.

## Running the Models with FBA

Now that we have the `manifest`, we can load the model as a `Community` object through `micom`. This will give us some functionality similar to that of `cobrapy`. This can be done with the `load_pickle()` method we imported above through `micom`.

In [2]:
from binary_models.load_av_rt import load_community_model, model_folder
community, manifest = load_community_model(prep_files=False)

Set parameter Username
Set parameter GURO_PAR_SPECIAL
Set parameter TokenServer to value "leghorn"


Output()

Output()

Read LP format model from file C:\Users\pino216\AppData\Local\Temp\tmpsomtxuxt.lp
Reading time = 0.05 seconds
: 4554 rows, 10733 columns, 41547 nonzeros


### Exploring the Model Attributes

Our new variable `community` behaves very similarly to a standard `cobrapy` model. We can explore it's attributes in a similar way as well.

Things such as `reactions` and `metabolites`:

In [ ]:
#community.reactions

In [ ]:
#community.metabolites

and importantly the `medium`

In [3]:
community.medium

{'EX_pi_m': 999999.0,
 'EX_h_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_mn2_m': 999999.0,
 'EX_fe2_m': 999999.0,
 'EX_glc__D_m': 5.0,
 'EX_zn2_m': 999999.0,
 'EX_acnam_m': 999999.0,
 'EX_fuc__L_m': 999999.0,
 'EX_rmn_m': 999999.0,
 'EX_galct__D_m': 999999.0,
 'EX_tartr__L_m': 999999.0,
 'EX_glcur_m': 999999.0,
 'EX_mg2_m': 999999.0,
 'EX_lcts_m': 999999.0,
 'EX_galctn__D_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_all__D_m': 999999.0,
 'EX_ni2_m': 999999.0,
 'EX_cu2_m': 999999.0,
 'EX_cobalt2_m': 999999.0,
 'EX_23camp_m': 999999.0,
 'EX_23cgmp_m': 999999.0,
 'EX_23cump_m': 999999.0,
 'EX_3amp_m': 999999.0,
 'EX_3gmp_m': 999999.0,
 'EX_3ump_m': 999999.0,
 'EX_sel_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_glcr_m': 999999.0,
 'EX_mnl_m': 999999.0,
 'EX_nh4_m': 999999.0,
 'EX_mobd_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_k_m': 999999.0,
 'EX_na1_m': 999999.0,
 'EX_thrp_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_23ccmp_m': 999999.0,
 'EX_3cmp_m': 999999.0,
 'EX_tyrp_m': 999999.0,
 'EX_cl_m': 99

This behavior mimics the medium in `cobrapy`, but combines both models mediums into 1

### Running Optimization

Now that we have the model loaded, we can run standard `FBA` methods using `optimize()`. Default optimize does not return any fluxes from the model, so we can set the `fluxes=True` when calling the method to return them.

In [4]:
community.abundances

id
Azotobacter       0.5
Rhodosporidium    0.5
Name: abundance, dtype: object

In [5]:
result = community.optimize(fluxes=True, pfba=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.5,42.896263,2492,2024
Rhodosporidium,0.5,1.335729,2423,2076
medium,NaN,NaN,451,451


We can see that both organisms have a non-zero growth rate and that the community growth is also non-zero. Let's check the fluxes.

In [6]:
result.fluxes.T.loc['EX_glyc__R_e']

compartment
Azotobacter      -100.0
Rhodosporidium    100.0
medium              NaN
Name: EX_glyc__R_e, dtype: float64

### Testing

In [7]:
#community.reactions.BIOMASS_Av_DJ_core__Azotobacter.upper_bound
#community.reactions.BIOMASS_RT__Rhodosporidium.upper_bound
#community.reactions.BIOMASS_Av_DJ_core__Azotobacter

In [8]:
community.set_abundance([1,1], normalize=False)

In [9]:
community.optimize(fluxes=True, pfba=True)

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,1.0,42.732957,2492,2024
Rhodosporidium,1.0,1.335798,2423,2076
medium,NaN,NaN,451,451


#### Changing parts of the medium to test it's effect on growth

Now that we can successfully optimize the community model, we can begin altering the models medium and seeing how it changes the (community) growth rate.

First, let's make a copy of the original medium so that we can restore it after making changes.

In [10]:
medium_bkp = community.medium

Now we can make changes to the medium. The following cell is meant to be re-run with making changes. It will first restore the medium to the original and them set 

In [11]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_xyl__D_m"] = 0
medium_to_change["EX_glc__D_m"] = 5
#medium_to_change["EX_glyc__R_m"] = 0
medium_to_change["EX_nh4_m"] = 0
medium_to_change["EX_n2_m"] = 5


# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

{'EX_pi_m': 999999.0,
 'EX_h_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_mn2_m': 999999.0,
 'EX_fe2_m': 999999.0,
 'EX_glc__D_m': 5,
 'EX_zn2_m': 999999.0,
 'EX_acnam_m': 999999.0,
 'EX_fuc__L_m': 999999.0,
 'EX_rmn_m': 999999.0,
 'EX_galct__D_m': 999999.0,
 'EX_tartr__L_m': 999999.0,
 'EX_glcur_m': 999999.0,
 'EX_mg2_m': 999999.0,
 'EX_lcts_m': 999999.0,
 'EX_galctn__D_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_all__D_m': 999999.0,
 'EX_ni2_m': 999999.0,
 'EX_cu2_m': 999999.0,
 'EX_cobalt2_m': 999999.0,
 'EX_23camp_m': 999999.0,
 'EX_23cgmp_m': 999999.0,
 'EX_23cump_m': 999999.0,
 'EX_3amp_m': 999999.0,
 'EX_3gmp_m': 999999.0,
 'EX_3ump_m': 999999.0,
 'EX_sel_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_glcr_m': 999999.0,
 'EX_mnl_m': 999999.0,
 'EX_mobd_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_k_m': 999999.0,
 'EX_na1_m': 999999.0,
 'EX_thrp_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_23ccmp_m': 999999.0,
 'EX_3cmp_m': 999999.0,
 'EX_tyrp_m': 999999.0,
 'EX_cl_m': 999999.0,
 'EX_tungs_m': 99

Now that the medium is changed, we can rerun the model optimization.

In [12]:
result_altered_medium = community.optimize(fluxes=True, pfba=True)

In [13]:
result_altered_medium

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,1.0,42.732957,2492,2024
Rhodosporidium,1.0,1.335798,2423,2076
medium,NaN,NaN,451,451


In [14]:
community.reactions.EX_glyc__R_e__Rhodosporidium

Reaction identifier,EX_glyc__R_e__Rhodosporidium
Name,(R)-Glycerate exchange
Memory address,0x020d9f4a4040
Stoichiometry,glyc__R_e__Rhodosporidium <=> glyc__R_m (R)-Glycerate <=> (R)-Glycerate
GPR,
Lower bound,-100
Upper bound,999999.0


In [15]:
community.reactions.EX_glyc__R_e__Azotobacter

Reaction identifier,EX_glyc__R_e__Azotobacter
Name,(R)-Glycerate exchange
Memory address,0x020d9aad6950
Stoichiometry,glyc__R_e__Azotobacter <=> glyc__R_m (R)-Glycerate <=> (R)-Glycerate
GPR,
Lower bound,-100
Upper bound,999999.0


In [16]:
result_altered_medium.fluxes.T.loc['EX_glyc__R_e']

compartment
Azotobacter      -100.0
Rhodosporidium    100.0
medium              NaN
Name: EX_glyc__R_e, dtype: float64

In [17]:
result_altered_medium.fluxes.T.loc['EX_glc__D_e']

compartment
Azotobacter      -100.0
Rhodosporidium     95.0
medium              NaN
Name: EX_glc__D_e, dtype: float64

In [18]:
result_altered_medium.fluxes.T.loc['EX_co2_m']

compartment
Azotobacter               NaN
Rhodosporidium            NaN
medium            2410.734729
Name: EX_co2_m, dtype: float64

In [19]:
x = result_altered_medium.fluxes.T.loc[result_altered_medium.fluxes.T.Azotobacter.index.str.startswith('EX_')]

In [20]:
from IPython.display import HTML

In [21]:
x.fillna(0,inplace=True)
HTML(x[(x.medium != 0) | (x.medium != 0)].sort_values('medium').to_html())

compartment,Azotobacter,Rhodosporidium,medium
reaction,,,
EX_h2o_m,0.0,0.0,-200.000000
EX_o2_m,0.0,0.0,-200.000000
EX_tre_m,0.0,0.0,-200.000000
EX_so4_m,0.0,0.0,-100.185376
EX_rmn_m,0.0,0.0,-100.000000
EX_galctn__D_m,0.0,0.0,-100.000000
EX_galct__D_m,0.0,0.0,-100.000000
EX_thrp_m,0.0,0.0,-100.000000
EX_hom__L_m,0.0,0.0,-100.000000


In [22]:
x.fillna(0,inplace=True)
HTML(x[(x.Azotobacter != 0) | (x.Rhodosporidium != 0)].sort_values('Azotobacter').to_html())

compartment,Azotobacter,Rhodosporidium,medium
reaction,,,
EX_rmn_e,-100.000000,0.000000,0.0
EX_galctn__D_e,-100.000000,0.000000,0.0
EX_so3_e,-100.000000,100.000000,0.0
EX_thrp_e,-100.000000,0.000000,0.0
EX_glc__D_e,-100.000000,95.000000,0.0
EX_glcur_e,-100.000000,0.000000,0.0
EX_tre_e,-100.000000,-100.000000,0.0
EX_sbt__D_e,-100.000000,100.000000,0.0
EX_galct__D_e,-100.000000,0.000000,0.0


In [23]:
from IPython.display import HTML

### Testing Abundances

The following code is an inline way to change the abundances

In [24]:
community.set_abundance([1,1],normalize=False)

## Running the Models with MICOM Grow

An alternative to running standard community optimization with `optimize()`, we can also use a `micom.workflows` method called `grow()`. This simulates growth of the organism while also simulating potential tradeoffs (between prioritizing community vs. individual growth). This method does not require our previously constructed `community` object, but rather the `manifest` we added earlier.

A key difference here though, is that we need to create a `DataFrame` detailing the reaction, flux, and metabolite as the medium provided to the method.

### Building the Medium

In [25]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
grow_medium_to_change = community.medium

#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
#grow_medium_to_change["EX_sucr_m"] = 1
grow_medium_to_change["EX_nh4_m"] = 0
grow_medium_to_change["EX_n2_m"] = 5

In [26]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

,reaction,flux
0,EX_pi_m,999999.0
1,EX_h_m,999999.0
2,EX_fe3_m,999999.0
3,EX_mn2_m,999999.0
4,EX_fe2_m,999999.0
...,...,...
65,EX_tyr__L_m,999999.0
66,EX_agm_m,999999.0
67,EX_acgal_m,999999.0
68,EX_gam_m,999999.0


In [ ]:
result_grow = grow(
    manifest, 
    model_folder=model_folder,
    medium=grow_medium, 
    tradeoff=0.01, 
    threads=2,
    presolve=True
)

Output()

In [ ]:
result_grow.exchanges

In [ ]:
result_grow.exchanges.to_csv('av_se_out.csv')